# Detectability Map
@author: Max Felius

Detectability map using a 1x1 km subset of track 88 Sentinel-1 data.

The goal is to test the subset on every position to determine if it is possible to detect a particular sinkhole (retrieved from digitised sinkhole analysis) if it would happen at that particular position. 

In [ ]:
#imports
import numpy as np
import pandas as pd
import sys, os, time
import matplotlib.pyplot as plt
from tqdm import tqdm

#import self created package
# from Detectability_Map.Detectability_Map_Creator import detectability_map
from Detectability_Map.read_sentinel1_csv_data import dataset
from Detectability_Map.rijksdriehoek import Rijksdriehoek

In [ ]:
folder = 'Subsets'
filename = 'subset_r400_lon6.06_lat50.87.csv'
dataset_filename = os.path.join(folder,filename)

dataset_obj = dataset(dataset_filename)

extend_rd = dataset_obj.extend_rd #[min x, max x, min y, max y]
extend_wgs = dataset_obj.extend_wgs #[min lon, max lon, min lat, max lat]

In [ ]:
#defining the different influence functions
def zg(R,r,itype='gaus'):
    # Automatically uses the Gaussian Influence Function unless specified differently
    if itype == 'gaus':
        return -zg_gaus(R,r) 
    elif itype == 'bals':
        return -zg_bals(R,r)    
    elif itype == 'beyer':
        return -zg_beyer(R,r)    
    else:
        print(f'Unknown itype: {itype}.')

def zg_gaus(R,r):
    return (1/(R*R))*np.exp(-np.pi*(r**2/R**2))

def zg_bals(H,r):
    r = np.sqrt((x-x0)**2 + (y-y0)**2)
    zone = np.arctan(r/H)
    return np.cos(zone)**2

def zg_beyer(R,r):
#     r = np.sqrt((x-x0)**2 + (y-y0)**2)\n",
    kz = ((3)/(np.pi*R**2))*(1-(r/R)**2)**2
    kz[r>R] = 0
    return kz

## Sinkhole Parameters

What would happen if a sinkhole, observered in the paper of Wink 2019, would happen in Limburg? Would we be able to detect it?

Wink 2019: <br>
Kim, J. W., Lu, Z., & Kaufmann, J. (2019). Evolution of sinkholes over Wink, Texas, observed by high-resolution optical and SAR imagery. Remote Sensing of Environment, 222, 119-132.

<strong>Parameters used</strong><br>
$v = 44.77 m^3/day$ <br>
$t = [44, 55, 66, 77, 88, 99, 110, 121, 132, 165]$ days

<strong>Methodology Checking Solution</strong>
- $v$ and $t$ are predefined using parameters found in the paper of Wink 2019.
- Since the model is nonlinear, the linearized design matix ($J$) is used.
- The design matrix ($J$) is set up to estimate $v$ and $R$.
- The size of the design matrix ($J$) is ($nm\thinspace x\thinspace 2$) where $n$ is the length of $\vec{r}$ and $m$ the length of $\vec{t}$.
- When the conditional number of the design matrix ($J$) is lower than $\frac{1}{\epsilon_{system}}$, a 1 is returned meaning that a solution is possible.

In [ ]:
def check_subset(subset,R,x0,y0,max_cond):
    '''
    Checks if a subset has a solvable design matrix. For now it only uses the Gaussian Influence Function.

    Input:
    :type subset: pandas dataframe
    :type R: int
    :type x0: int
    :type y0: int

    Output
    :rtype: int, float
    '''
    #sinkhole parameters for Wink 2019
    v = 44.77
    t = np.array([44, 55, 66, 77, 88, 99, 110, 121, 132, 165])
   
    itype = 'gaus'
    
    # print('Computing the radius.')
    r = np.sqrt((subset['pnt_rdx'].values-x0)**2 + (subset['pnt_rdy'].values-y0)**2)

    if len(r)==0:
        return 0, 0.0
    else:
        #defining the jacobian matrix for nonlinear least squares
        a = zg(R,r,itype).reshape((len(r),1))
        b = t.reshape((len(t),1))
        
        A1 = a @ b.T
        A1 = A1.ravel()
        
        c = (((2*R**2 + 2*np.pi*r**2)/(R**3))*(v)*zg(R,r,itype)).reshape((len(r),1))
        
        A2 = c @ b.T
        A2 = A2.ravel()
        
        J = np.array([A1,A2]).T
        cond_number = np.linalg.cond(J)

    if cond_number > max_cond:#1/sys.float_info.epsilon:
        return 0, cond_number
    else:
        return 1, cond_number

In [ ]:
def make_map(R_list,x_eval,y_eval,dataset_obj,max_cond):
    '''
    Method to create the detectability map. 
    
    Input are the evaluation coordinates and the list of the radius of infuence.

    Input:_pos
    :type R_list: list[int]
    :type x_eval: list[float], rd
    :type y_eval: list[float], rd
    :type dataset_obj: object(dataset)

    Output:
    :rtype pandas.DataFrame
    '''
    header_outputframe = ['pnt_lon','pnt_lat','pnt_rdx','pnt_rdy','geometry (wgs)','geometry (rd)','sum']
    
    #implement R columns in the header
    for item in R_list:
        header_outputframe.append(f'R_{item}')
        header_outputframe.append(f'Cond_R_{item}')
    
    # building the numbers for in the dataframe
    lon_eval_wgs, lat_eval_wgs = Rijksdriehoek(x_eval, y_eval).to_wgs()
    fill_data = np.zeros((len(lon_eval_wgs),len(header_outputframe)-4))
    coordinate_data = np.array([lon_eval_wgs,lat_eval_wgs,x_eval,y_eval]).T
    coordinate_data = np.concatenate((coordinate_data,fill_data),axis=1)
    
    # Building dataframe
    df = pd.DataFrame(coordinate_data,columns=header_outputframe)
    
    for idx,R in enumerate(R_list):
#         print(f'Set {idx+1}/{len(R_list)}.')
        for i in tqdm(range(len(x_eval)),desc=f'Making the Map ({idx+1}/{len(R_list)})'):
            #select center position
            x_pos = x_eval[i]
            y_pos = y_eval[i]

            #create the subset
            subset = dataset_obj.create_spatial_subset([x_pos,y_pos],R)

            result, cond = check_subset(subset,R,x_pos,y_pos,max_cond)

            #filling the dataframe
            #create a polygon of the entry
            lbrd = Rijksdriehoek(x_pos-(min(R_list)/4),y_pos-(min(R_list)/4))
            rbrd = Rijksdriehoek(x_pos+(min(R_list)/4),y_pos-(min(R_list)/4))
            rtrd = Rijksdriehoek(x_pos+(min(R_list)/4),y_pos+(min(R_list)/4))
            ltrd = Rijksdriehoek(x_pos-(min(R_list)/4),y_pos+(min(R_list)/4))

            leftbot = lbrd.to_wgs()
            rightbot = rbrd.to_wgs()
            righttop = rtrd.to_wgs()
            lefttop = ltrd.to_wgs()

            df['geometry (wgs)'].iloc[i] = f'POLYGON (({leftbot[1]} {leftbot[0]},{rightbot[1]} {rightbot[0]},{righttop[1]} {righttop[0]},{lefttop[1]} {lefttop[0]}))'
            df['geometry (rd)'].iloc[i] = f'POLYGON (({lbrd.rd_x} {lbrd.rd_y},{rbrd.rd_x} {rbrd.rd_y},{rtrd.rd_x} {rtrd.rd_y},{ltrd.rd_x} {ltrd.rd_y}))'

            df[f'R_{R}'].iloc[i] = result
            df[f'Cond_R_{R}'].iloc[i] = cond
            
            if result == 1:
                df['sum'].iloc[i] += 1
        
    return df

## Defining the Settings and Executing the map making

In [ ]:
max_cond_num = [1500,2000,10000,1000000]

    #The R_list vector determines which sinkhole sizes to evaluate. 
    #The resolution of the output map is taken from the lowest value in the R_list vector

for mcond in max_cond_num:
    # R_list = [1,2,3,4,5,6,8,9,10,20,30,40,50,60,70,80,90,100,200,300]
    # R_list = [30,40]
    R_list = [10,20,30,40,50,60,70,80,90,100,200,300]

    coordinates = extend_rd
    xmin = coordinates[0]
    xmax = coordinates[1]

    x_range = np.arange(xmin,xmax,min(R_list)/2)

    ymin = coordinates[2]
    ymax = coordinates[3]

    y_range = np.arange(ymin,ymax,min(R_list)/2)

    xv,yv = np.meshgrid(x_range,y_range)

    x_eval = xv.ravel()
    y_eval = yv.ravel()

    df = make_map(R_list,x_eval,y_eval,dataset_obj,mcond)

    df.to_csv(f'Detectability_Map_v3_cond{mcond}.csv')

## Save the results

In [ ]:
df.to_csv('Detectability_Map_v3.csv')